In [1]:
import pandas as pd
import os
import re
import numpy as np

from IPython.display import display
pd.options.display.max_columns = None

### Import Top 200 Chart data

In [2]:
# path = '/home/jovyan/work/Documents/Correlation One/spotipy/'
path = r'/Users/minhhoang/Documents/Correlation One/spotipy'
path_input = os.path.join(path, 'input')

In [3]:
# Import check point 1 (dataset includes all songs on charts regardless of collaboration, but does not have no. of colalborators)
data_top200 = pd.read_csv(os.path.join(path_input, 'spotify_top200.csv'))

# Import checkpoint 2 (dataset only includes songs with at least 2 collaborators)
data_top200_reduced = pd.read_csv(os.path.join(path_input, 'spotify_top200_reduced.csv'))\
.drop(columns = ['Unnamed: 0'])

In [4]:
data_top200_augment = data_top200.drop_duplicates()\
.merge(data_top200_reduced.drop_duplicates(), how='left', 
                            on=['Date', 'Track URL', 'Position', 'Track Name', 
                                'Artist', 'Streams', 'Country'])\
.drop_duplicates()

# Brunnell's input data has duplicated rows for US, UK and Global

In [6]:
print(len(data_top200.index))
print(len(data_top200.drop_duplicates().index))

print(len(data_top200_reduced.index))
print(len(data_top200_reduced.drop_duplicates().index))

14546258
14249658
6581714
6457503


In [7]:
print(len(data_top200_augment.index))

14249658


In [8]:
data_top200_augment['Country'] = data_top200_augment['Country'].apply(lambda x: 'Viet Nam' if x == 'Viet Name' else x) 
data_top200_augment['Date'] = pd.to_datetime(data_top200_augment['Date'])
data_top200_augment['Year'] = data_top200_augment['Date'].dt.strftime('%Y')

In [9]:
data2 = data_top200_augment.sort_values(by=['Country', 'Artist', 'Date', 'Track Name'], ascending=[True,True,True,True])

### Generate new variables

Calculate a song's earliest appearance on the chart

In [10]:
data3 = data2
data3['Song_min_date'] = data3.groupby(['Country', 'Artist', 'Track Name'])[['Date']].transform('min')

Calculate the number of days between the current chart day and the song's first day on the chart 

In [11]:
data3['Song_days_since_first'] = data3['Date'] - data3['Song_min_date']
data3['Song_days_since_first'] = data3['Song_days_since_first'].apply(lambda x: x.days)

Calculate the actual number of days that the song has appeared on the chart 

In [12]:
data3['Song_days_onchart'] = data3.groupby(['Country', 'Artist', 'Track Name']).cumcount() + 1
# The two previous variables are similar, but may differ if the song temporarily goes off the chart and gets back on at a later date

Identify the first time an artist appears on the Top 200 chart (in a country)
This needs to be run with the complete dataset, not just on songs with collaboration

In [13]:
data3['Artist_min_date'] = data3.groupby(['Country', 'Artist'])[['Date']].transform('min')

Generate a dummy variable that = 1 if this is the first time a song appears

In [14]:
data3['Artist_new_song'] = np.where(data3['Song_days_onchart']==1, 1, 0)

Calculate the number of days elapsed between a song's current appearance and its last appearance
If Date_diff = 1, then the song appeared yesterday and today.

In [15]:
data3['Date_lag'] =  data3.groupby(['Country', 'Artist', 'Track Name'])[['Date']].shift(1)
data3['Date_diff'] = data3['Date'] - data3['Date_lag']
data3['Date_diff'] = data3['Date_diff'].apply(lambda x: x.days)

When Date_diff is > 1 , we assume that the song has temporarily dropped off the chart (hence ending its last consecutive streak),
so we create a binary variable "Song_new_streak" that signals the beginnning of a new streak

In [16]:
data4 = data3
data4['Song_new_streak'] = np.where((data4['Song_days_onchart']==1)|(data4['Date_diff']>1), 1, 0)

For each country-artist-song's consecutive streak, assign a unique id by performing a cumulative sum of the 'Song_new_streak' variable

In [17]:
data4['Song_streak_id'] = data4.groupby(['Country', 'Artist', 'Track Name'])[['Song_new_streak']].transform('cumsum')

For each day within a streak, assign a unique id

In [18]:
data4['Song_consec_day'] = data4.groupby(['Country', 'Artist', 'Track Name', 'Song_streak_id']).cumcount() + 1

Calculate the song's cumulative total number of streams and cumulative average number of streams

In [19]:
data4['Song_cumu_streams'] = data4.groupby(['Country', 'Artist', 'Track Name'])[['Streams']].transform('cumsum')
data4['Song_cumu_mean_streams'] = data4['Song_cumu_streams'] / data4['Song_days_onchart']

Calculate an artist's cumulative number of songs that have appeared on the chart

In [20]:
data5 = data4
data5['Artist_cumsum_songs'] = data5.groupby(['Country', 'Artist'])[['Artist_new_song']].transform('cumsum')
data5['Artist_cumu_songs'] = data5.groupby(['Country', 'Artist', 'Date'])[['Artist_cumsum_songs']].transform('max')

Calculate an artist's cumulative number of days of appearing on the chart (not necessarily consecutively)

In [21]:
artist_days = data5.loc[:, ['Country', 'Artist', 'Date']]\
.sort_values(by=['Country', 'Artist', 'Date'], ascending=[True,True,True]).drop_duplicates()
artist_days['Artist_days_onchart'] = artist_days.groupby(['Country', 'Artist']).cumcount() + 1

artist_days

,Country,Artist,Date,Artist_days_onchart
8773894,Argentina,#TocoParaVos,2017-01-01,1
8806453,Argentina,#TocoParaVos,2017-01-02,2
11463480,Argentina,#TocoParaVos,2017-01-03,3
7710816,Argentina,#TocoParaVos,2017-01-04,4
8751390,Argentina,#TocoParaVos,2017-01-05,5
...,...,...,...,...
2685112,Viet Nam,NaN,2019-08-25,874
2685302,Viet Nam,NaN,2019-08-26,875
2685507,Viet Nam,NaN,2019-08-27,876
2685715,Viet Nam,NaN,2019-08-28,877


Calculate an artist's cumulative total number of streams and cumulative average number of streams

In [22]:
data6 = data5.merge(artist_days, how='left', on=['Country', 'Artist', 'Date'])

data6['Artist_cumsum_streams'] = data6.groupby(['Country', 'Artist'])[['Streams']].transform('cumsum')
data6['Artist_cumu_streams'] = data6.groupby(['Country', 'Artist', 'Date'])[['Artist_cumsum_streams']].transform('max')

# Needs more reflection for this variable - cumulative averages are not accurate because we only observe the song if it's in the top 200
# if the song is not in the top 200, then the cumulative number of streams cannot be updated
# data6['Artist_cumu_mean_streams'] = data6['Artist_cumu_streams'] / data6['Artist_days_onchart']

Generate a unique id for each week and year-week 
(this will be used to merge with other datasets that only update on a weekly basis, e.g. Billboard)

In [23]:
data6['Week_id'] = data6['Date'].dt.strftime('%W')
data6['Year_week_id'] = data6['Date'].dt.strftime('%Y-%W')
data6['Day_week'] = data6['Date'].dt.strftime('%a')

# Create a separate week id to merge with Billboard data, which resets on a Tuesday instead of Monday
data6['Week_id_BB'] = np.where(data6['Day_week'] == 'Mon', 
                               data6['Week_id'].astype(int) - 1, data6['Week_id'].astype(int))

# Calculate the mean number of streams for each week id in a country
data6['Week_mean_streams'] = data6.groupby(['Country', 'Year_week_id'])[['Streams']].transform('mean')

In [ ]:
data6.head()

In [24]:
data7 = data6[data6['No. of Collaborators'].notna()]

In [27]:
print(len(data7[data7['No. of Collaborators'].isna()]))
print(len(data7[data7.duplicated()]))
print(data7['Year'].unique())

0
0


In [29]:
data8 = data7[data7['Country'] == 'United States']

In [31]:
print(len(data2.index))
print(len(data3.index))
print(len(data4.index))
print(len(data5.index))
print(len(data6.index))
print(len(data7.index))
print(len(data8.index))

14249658
14249658
14249658
14249658
14249658
6457503
116428


In [34]:
data7.loc[(data6['Artist']=='Drake') &
          (data6['Country']=='United States') &
          (data6['Track Name']=='One Dance')
          ,:]\
.drop(['Track URL', 'Track URI', 'Collaborator URI'], axis=1).head()

,Date,Position,Track Name,Artist,Streams,Country,No. of Collaborators,Year,Song_min_date,Song_days_since_first,Song_days_onchart,Artist_min_date,Artist_new_song,Date_lag,Date_diff,Song_new_streak,Song_streak_id,Song_consec_day,Song_cumu_streams,Song_cumu_mean_streams,Artist_cumsum_songs,Artist_cumu_songs,Artist_days_onchart,Artist_cumsum_streams,Artist_cumu_streams,Week_id,Year_week_id,Day_week,Week_id_BB,Week_mean_streams
13761259,2017-01-01,7,One Dance,Drake,753150,United States,3.0,2017,2017-01-01,0.0,1,2017-01-01,1,NaT,NaN,1,1,1,753150,753150.000000,5,9.0,1,2778807,3909721.0,00,2017-00,Sun,0,275695.290000
13761269,2017-01-02,13,One Dance,Drake,487243,United States,3.0,2017,2017-01-01,1.0,2,2017-01-01,0,2017-01-01,1.0,0,1,2,1240393,620196.500000,10,10.0,2,6293003,7296288.0,01,2017-01,Mon,0,272030.682857
13761279,2017-01-03,18,One Dance,Drake,512037,United States,3.0,2017,2017-01-01,2.0,3,2017-01-01,0,2017-01-02,1.0,0,1,3,1752430,584143.333333,10,10.0,3,9856908,10974059.0,01,2017-01,Tue,1,272030.682857
13761289,2017-01-04,20,One Dance,Drake,498653,United States,3.0,2017,2017-01-01,3.0,4,2017-01-01,0,2017-01-03,1.0,0,1,4,2251083,562770.750000,10,10.0,4,13524121,14640932.0,01,2017-01,Wed,1,272030.682857
13761299,2017-01-05,19,One Dance,Drake,504230,United States,3.0,2017,2017-01-01,4.0,5,2017-01-01,0,2017-01-04,1.0,0,1,5,2755313,551062.600000,10,10.0,5,17232069,18405690.0,01,2017-01,Thu,1,272030.682857


In [ ]:
# data6.loc[(data6['Artist']=='Drake') &
#           (data6['Country']=='United States')
#           ,:]\
# .drop(['Track URL'], axis=1).head(25)

In [ ]:
# data6.to_csv('2020.12.14 top200_clean.csv')

In [ ]:
# data7.to_csv('2020.12.15 top200_reduced_fe.csv', index=False)

In [ ]:
# data8.to_csv('2020.12.15 top200_reduced_fe_usa.csv', index=False)

In [ ]:
# data6[data6['Country']=='United States'].to_csv('2020.12.14 top200_usa_clean.csv')